<a href="https://www.kaggle.com/code/abd0kamel/2023-10-sprint2-kaggle?scriptVersionId=148289293" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# WLASL Dataset Sign Language Model

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
# #         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# from IPython.core.display import clear_output
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import json

# Load the JSON file
with open('/kaggle/input/wlasl-processed/WLASL_v0.3.json', 'r') as json_file:
    data = json.load(json_file)

# Words to filter by
words_to_filter = ["where","hello","thank you", "go","drop","here","traffic","good","bad","today"]

# Filter the data
filtered_data = [item for item in data if item.get("gloss") in words_to_filter]

# Extract the first 5 "bbox" elements from each item
for item in filtered_data:
    item["instances"] = item["instances"][:5]

# Save the filtered data with the first 5 "bbox" elements to a JSON file
with open('filtered_data_with_5_bbox.json', 'w') as output_file:
    json.dump(filtered_data, output_file, indent=4)



In [3]:
import os

input_folder = '/kaggle/input/wlasl-processed/videos'

In [4]:
video_files = [f for f in os.listdir(input_folder) if f.endswith('.mp4')]
print(len(video_files)) 

11980


In [5]:
with open('/kaggle/working/filtered_data_with_5_bbox.json', 'r') as f:
    data = json.load(f)

In [78]:
gloss_list = [] # done
video_name_list = [] # done
start_frame = [] # done
end_frame = [] # done
vid_dir = '/kaggle/input/wlasl-processed/videos'
progress = 0

for word in data:
    for i in range(5):
        vid_name = os.path.join(vid_dir, f'{word["instances"][i]["video_id"]}.mp4')
        start = word['instances'][i]['frame_start']
        end = word['instances'][i]['frame_end']
        label = word['gloss']

        video_name_list.append(vid_name)
        gloss_list.append(label)
        start_frame.append(start)
        end_frame.append(end)

        progress += 1
        print(f'Progress: {progress} / {len(data)*5}')

Progress: 1 / 50
Progress: 2 / 50
Progress: 3 / 50
Progress: 4 / 50
Progress: 5 / 50
Progress: 6 / 50
Progress: 7 / 50
Progress: 8 / 50
Progress: 9 / 50
Progress: 10 / 50
Progress: 11 / 50
Progress: 12 / 50
Progress: 13 / 50
Progress: 14 / 50
Progress: 15 / 50
Progress: 16 / 50
Progress: 17 / 50
Progress: 18 / 50
Progress: 19 / 50
Progress: 20 / 50
Progress: 21 / 50
Progress: 22 / 50
Progress: 23 / 50
Progress: 24 / 50
Progress: 25 / 50
Progress: 26 / 50
Progress: 27 / 50
Progress: 28 / 50
Progress: 29 / 50
Progress: 30 / 50
Progress: 31 / 50
Progress: 32 / 50
Progress: 33 / 50
Progress: 34 / 50
Progress: 35 / 50
Progress: 36 / 50
Progress: 37 / 50
Progress: 38 / 50
Progress: 39 / 50
Progress: 40 / 50
Progress: 41 / 50
Progress: 42 / 50
Progress: 43 / 50
Progress: 44 / 50
Progress: 45 / 50
Progress: 46 / 50
Progress: 47 / 50
Progress: 48 / 50
Progress: 49 / 50
Progress: 50 / 50


In [97]:
print(f'number of words = {len(gloss_list)}\nnumber of videos = {len(video_name_list)}')

number of words = 50
number of videos = 50


In [98]:
import numpy as np

# Specify the file path to your .npy file
file_path = "/kaggle/input/opencvtest/thanks.npy"

# Load the .npy file
loaded_data = np.load(file_path)


In [99]:
loaded_data.shape

(90, 543, 3)

In [100]:
import cv2
import numpy as np

def draw_landmarks(input_path, output_path, npy_file, start_frame=0, end_frame=-1):
    """
    Reads a video from the input file, overlays landmarks on each frame, and saves the result to an output video file.

    Args:
        input_path (str): The path to the input video file.
        output_path (str): The path to save the output video with overlaid facial landmarks.
        npy_file (str): The path to a NumPy file containing facial landmarks data for each frame.
        start_frame (int): The index of the starting frame for landmark overlay (default is 0).
        end_frame (int): The index of the ending frame for landmark overlay (default is -1, meaning the last frame).

    Description:
        This function reads a video from the input file, extracts facial landmarks data from a NumPy file,
        and overlays landmarks on each frame of the video. The frames within the specified range,
        from 'start_frame' (inclusive) to 'end_frame' (exclusive), are processed. Facial landmarks are drawn as
        red circles on the face, hands, and body in each frame. The output video is saved to the 'output_path'
        with the same resolution and frame rate as the input video.
    """

    cap = cv2.VideoCapture(input_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    landmarks_data = np.load(npy_file)
    frame_index = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_index >= start_frame and frame_index < end_frame:
            landmarks_frame = landmarks_data[frame_index - start_frame]
            landmarks = [(int(x * width), int(y * height)) for x, y, z in landmarks_frame]
            for x, y in landmarks:
                cv2.circle(frame, (x, y), 3, (0, 0, 255), -1)
            out.write(frame)
        else:
            # out.write(frame) # Enable if you want the full video
            pass
        frame_index += 1

    cap.release()
    out.release()


In [69]:
! pip -q install mediapipe
import mediapipe as mp

In [101]:
hands = mp.solutions.hands.Hands()
pose = mp.solutions.pose.Pose()
face_mesh = mp.solutions.face_mesh.FaceMesh()

In [102]:
def get_frame_landmarks(frame):
    """
    Extracts landmarks from a single video frame using MediaPipe.

    Args:
        frame: A single rgb frame/image.

    Returns:
        np.array: A NumPy array containing extracted landmarks.
        The output dimensions are (n, 3) array, where n is the number of landmarks.
        Each row in the array represents a landmark, and each landmark is represented
        as [x, y, z], where x, y, and z are the normalized coordinates of the landmark.
    """

    results_hands = hands.process(frame)
    results_pose = pose.process(frame)
    results_face = face_mesh.process(frame)

    landmarks_per_hand = 21
    landmarks_body_pose = 33
    landmarks_face = 468         # Max 468

    all_landmarks = np.empty((landmarks_per_hand * 2 + landmarks_body_pose + landmarks_face, 3))

    if results_hands.multi_hand_landmarks:
        all_landmarks[:landmarks_per_hand, :] = np.array([(lm.x, lm.y, lm.z) for lm in results_hands.multi_hand_landmarks[0].landmark])
        if len(results_hands.multi_hand_landmarks) > 1:
            all_landmarks[landmarks_per_hand:landmarks_per_hand * 2, :] = np.array(
                [(lm.x, lm.y, lm.z) for lm in results_hands.multi_hand_landmarks[1].landmark])

    if results_pose.pose_landmarks:
        all_landmarks[landmarks_per_hand * 2:landmarks_per_hand * 2 + landmarks_body_pose, :] = np.array(
            [(lm.x, lm.y, lm.z) for lm in results_pose.pose_landmarks.landmark])

    if results_face.multi_face_landmarks:
        # all_landmarks[landmarks_per_hand * 2 + landmarks_body_pose:, :] = np.array(
            # [(lm.x, lm.y, lm.z) for lm in results_face.multi_face_landmarks[0].landmark[::468 // landmarks_face]]
            # )

        all_landmarks[landmarks_per_hand * 2 + landmarks_body_pose:, :] = np.array(
            [(lm.x, lm.y, lm.z) for lm in results_face.multi_face_landmarks[0].landmark]
            )

    return all_landmarks

In [103]:
def get_video_landmarks(video_path, start_frame=0, end_frame=-1, num_landmarks=543):
    """
    Extracts landmarks from a video by processing each frame in the video.

    Args:
        video_path (str): The file path to the video to process.
        start_frame (int): The index of the starting frame (default is 0).
        end_frame (int): The index of the ending frame (default is -1, meaning the last frame).

    Returns:
        np.array: A NumPy array where each row corresponds to the landmarks
        extracted from a single frame of the video within the specified frame range.
        The dimensions of the output array are (m, n, 3), where m is the number of frames
        within the specified range and n is the number of landmarks.
        Each element in the array is a 3D coordinate representing a landmark's position.
    """

    cap = cv2.VideoCapture(video_path)
    if end_frame < 0:
        end_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if start_frame < 0:
        start_frame = 0
    if end_frame < start_frame:
        start_frame, end_frame = end_frame, start_frame
        

    all_frame_landmarks = np.empty((end_frame - start_frame, num_landmarks, 3))
    frame_index = 0

    while cap.isOpened() and frame_index != end_frame:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_index >= start_frame:
            frame.flags.writeable = False
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_landmarks = get_frame_landmarks(frame)
            all_frame_landmarks[frame_index - start_frame] = frame_landmarks

        frame_index += 1

    cap.release()

    return all_frame_landmarks

In [104]:
lm = get_video_landmarks(video_name_list[0], start_frame=start_frame[0], end_frame=end_frame[0])

In [105]:
np.save('/kaggle/working/test.npy',lm)

In [106]:
np.sum(lm!=0)

1704

In [111]:
lm[0]

array([[4.04738577e-320, 3.55727265e-322, 0.00000000e+000],
       [4.87994243e-310, 4.94065646e-324, 4.87994243e-310],
       [4.87994214e-310, 4.94065646e-324, 4.87994243e-310],
       ...,
       [5.58294180e-321, 0.00000000e+000, 1.36362118e-321],
       [4.87994243e-310, 5.58788245e-321, 0.00000000e+000],
       [1.36362118e-321, 4.87994243e-310, 5.59282311e-321]])

In [ ]:
import os
input_video = '/kaggle/input/opencvtest/mCjHYreiZ24.mp4'
output_video = '/kaggle/working/ahhhhhhhh.mp4'
npy_path = "/kaggle/working/numpyOut/0.npy"

draw('/kaggle/input/wlasl-processed/videos/00335.mp4', output_video, '/kaggle/input/opencvtest/thanks.npy',start_frame=start_frame[0],end_frame=end_frame[0])

In [ ]:
video_name_list[0]